### 1. Отобразите все записи из таблицы company по компаниям, которые закрылись.

In [ ]:
SELECT *
FROM company
WHERE status = 'closed' ;

### 2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.

In [ ]:
SELECT funding_total
FROM company
WHERE country_code = 'USA' AND category_code = 'news'
ORDER BY funding_total DESC;

### 3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount) AS total_cash_deals
FROM acquisition
WHERE term_code = 'cash' 
      AND acquired_at 
BETWEEN '2011-01-01' AND '2013-12-31';

### 4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE  twitter_username LIKE 'Silver%';

### 5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%' AND last_name LIKE 'K%';


### 6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code, SUM(funding_total) AS total_investments
FROM company
GROUP BY country_code
ORDER BY total_investments DESC;


### 7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at, 
       MIN(raised_amount) AS min_investment, 
       MAX(raised_amount) AS max_investment
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0 
AND MIN(raised_amount) <> MAX(raised_amount);

### 8. Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
  CASE
    WHEN invested_companies >= 100 THEN 'high_activity'
    WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
    ELSE 'low_activity'
  END AS category
FROM fund;

### 9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT 
  category,
  ROUND(AVG(investment_rounds)) AS average_investment_rounds
FROM (
  SELECT *,
    CASE
      WHEN invested_companies >= 100 THEN 'high_activity'
      WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
      ELSE 'low_activity'
    END AS category
  FROM fund
) AS categorized_funds
GROUP BY category
ORDER BY average_investment_rounds;

### 10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM 
(SELECT *
FROM fund       
WHERE EXTRACT (YEAR FROM founded_at) BETWEEN 2010 AND 2012) AS f
 
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10;

### 11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT pep.first_name,
       pep.last_name,
       ed.instituition
FROM people AS pep
LEFT JOIN education AS ed ON pep.id=ed.person_id

### 12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
INNER JOIN people AS p ON c.id=p.company_id
INNER JOIN education AS e ON p.id=e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5

### 13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company AS c
LEFT JOIN funding_round AS fr ON c.id=fr.company_id
WHERE status LIKE '%closed%'
  AND is_first_round = 1
  AND is_last_round = 1

In [ ]:
SELECT DISTINCT c.name
FROM company AS c
WHERE c.id IN (
  SELECT fr.company_id
  FROM funding_round AS fr
  WHERE fr.is_first_round = 1
    AND fr.is_last_round = 1
)
AND c.status LIKE '%closed%';

### 14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN (
  SELECT com.id
  FROM company AS com
  INNER JOIN funding_round AS fr ON com.id = fr.company_id
  WHERE com.status LIKE '%closed%'
    AND fr.is_first_round = 1
    AND fr.is_last_round = 1
);

In [ ]:
SELECT DISTINCT p.id
FROM company AS com
INNER JOIN funding_round AS fr ON com.id=fr.company_id
INNER JOIN people AS p ON com.id=p.company_id
WHERE STATUS LIKE '%closed%'
  AND is_first_round = 1
  AND is_last_round = 1

### 15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT p.id, ed.instituition
FROM people AS p
INNER JOIN education AS ed ON p.id = ed.person_id
WHERE p.company_id IN (
  SELECT com.id
  FROM company AS com
  INNER JOIN funding_round AS fr ON com.id = fr.company_id
  WHERE com.status LIKE '%closed%'
    AND fr.is_first_round = 1
    AND fr.is_last_round = 1
);

### 16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT DISTINCT p.id,
       COUNT(ed.instituition)
FROM company AS com
INNER JOIN people AS p ON com.id=p.company_id
LEFT JOIN education AS ed ON p.id=ed.person_id
WHERE STATUS LIKE '%closed%'
  AND com.id IN (SELECT company_id
                FROM funding_round
                 WHERE is_first_round = 1
                   AND is_last_round = 1)
AND ed.instituition IS NOT NULL
GROUP BY p.id

### 17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(education_count) AS average_education_count
FROM (
  SELECT COUNT(ed.instituition) AS education_count
  FROM company AS com
  INNER JOIN people AS p ON com.id = p.company_id
  LEFT JOIN education AS ed ON p.id = ed.person_id
  WHERE com.id IN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1
      AND is_last_round = 1
  )
  AND com.status LIKE '%closed'
  AND ed.instituition IS NOT NULL
  GROUP BY p.id
) AS subquery;

### 18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook.

In [ ]:
SELECT AVG(education_count) AS average_education_count
FROM (
  SELECT COUNT(ed.instituition) AS education_count
  FROM company AS com
  INNER JOIN people AS p ON com.id = p.company_id
  LEFT JOIN education AS ed ON p.id = ed.person_id
  WHERE com.name = 'Facebook' 
  AND ed.instituition IS NOT NULL
  GROUP BY p.id
) AS subquery;

### 19. Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
INNER JOIN company AS c ON i.company_id = c.id
INNER JOIN fund AS f ON i.fund_id = f.id
INNER JOIN funding_round AS fr ON i.funding_round_id=fr.id
WHERE c.milestones > 6
  AND fr.funded_at BETWEEN '2012-01-01' AND '2013-12-31';

### 20. Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT
  acq_company.name AS acquiring_company_name,
  acq.price_amount AS acquisition_amount,
  acquired_company.name AS acquired_company_name,
  acquired_company.funding_total AS investment_amount,
  ROUND(acq.price_amount / NULLIF(acquired_company.funding_total, 0), 0) AS ratio
FROM acquisition AS acq
INNER JOIN company AS acq_company ON acq.acquiring_company_id = acq_company.id
INNER JOIN company AS acquired_company ON acq.acquired_company_id = acquired_company.id
WHERE acq.price_amount > 0
AND acquired_company.funding_total > 0
ORDER BY acq.price_amount DESC, acquired_company_name
LIMIT 10;

### 21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

SELECT
  c.name AS company_name,
  DATE_PART('month', f.funded_at) AS funding_month
FROM company AS c
INNER JOIN funding_round AS f ON c.id = f.company_id
WHERE c.category_code = 'social'
  AND f.funded_at BETWEEN '2010-01-01' AND '2013-12-31'
  AND f.raised_amount != 0;

### 22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
fundings AS (SELECT EXTRACT (MONTH FROM CAST(fr.funded_at AS DATE)) AS funding_month,
      COUNT(DISTINCT f.id) AS id_fund
FROM fund AS f     
LEFT JOIN investment AS i ON f.id=i.fund_id
LEFT JOIN funding_round AS fr ON i.funding_round_id=fr.id
WHERE f.country_code = 'USA'
  AND EXTRACT(YEAR FROM CAST (fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
GROUP BY funding_month),
 
acquisitions AS (SELECT EXTRACT (MONTH FROM CAST(acquired_at AS DATE)) AS funding_month,
      COUNT(acquired_company_id) AS acquired,
      SUM(price_amount) AS sum_total
FROM acquisition
WHERE EXTRACT(YEAR FROM CAST (acquired_at AS DATE)) BETWEEN 2010 AND 2013
GROUP BY funding_month) 
SELECT fnd.funding_month,
       fnd.id_fund,
       acq.acquired,
       acq.sum_total
FROM fundings AS fnd 
LEFT JOIN acquisitions AS acq ON fnd.funding_month=acq.funding_month

### 23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
a AS (SELECT country_code,
      AVG(funding_total) AS totalavg_2011
   FROM company
   WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2011
     GROUP BY country_code),
b AS (SELECT country_code,
      AVG(funding_total) AS totalavg_2012
   FROM company
   WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2012
     GROUP BY country_code),
c AS (SELECT country_code,
      AVG(funding_total) AS totalavg_2013
   FROM company
   WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2013
     GROUP BY country_code)
SELECT a.country_code,
       a.totalavg_2011,
       b.totalavg_2012,
       c.totalavg_2013
FROM a INNER JOIN b ON a.country_code = b.country_code INNER JOIN c ON a.country_code = c.country_code
ORDER BY totalavg_2011 DESC